# Inventa: a computational tool to discover structural novelty in natural  extracts libraries


In [1]:
from __future__ import print_function
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import sys 
import lineup_widget
from ipywidgets import *
!jupyter nbextension enable --py --sys-prefix lineup_widget

sys.path.append('../src')
sys.path.append('../gnps_postprocessing/src') 

from import_data import*
from process_data import *
from AC import *
from LC import *
from SC import *
from CC import *
from plot import *

Enabling notebook extension lineup_widget/extension...
      - Validating: OK


# Paths and parameters to define

In [2]:
# Suffixes necessary for the job: 

repository_path= '/mnt/c/Users/quirosgu/Desktop/Indiv_PF1600/'  # The path were you want your folder to be placed
#quant_table_suffix ='_quant_pos.csv'
spectra_suffix= '_features_ms2_pos.mgf'
#metadata_sample_suffix ='_metadata.tsv'
#isdb_sample_suffix = '_isdb_matched_pos_repond.tsv'
#sirius_sample_suffix = 'compound_identifications_adducts.tsv'
canopus_sample_suffix = 'npc_summary.csv' #'_summary_adducts.tsv' #'canopus_summary.tsv'
#memo_sample_suffix= '_memo_pos.csv'
file_extention = '.mzXML'
ionization_mode = 'pos'

# metadata headers

sampletype_header = 'sample_type'
species_column = 'organism_species'
genus_column = 'organism_genus'
family_column = 'organism_family'
filename_header = 'sample_id'#'ms_filename'
organe_column = 'organism_organe'


#quantitative table
data_process_origin = 'MZMine2' #'MZMine2'
use_ion_identity= False  #False

#Annotation component 

intensity_filter  = True
quantile_filter = True

min_threshold = 0.002
quantile_threshold = 0.75


## cut-offs: 
min_score_final = 0.3             #cut-off filter for considering an isdb annotation valable. You must be extremenly carefull with this parameter, '0.0' as default.
min_ZodiacScore = 0.9             #cut-off filter for considering a sirius annotation valable. It is used in combination with min_ConfidenceScore.
min_ConfidenceScore= 0.25         #cut-off filter for considering a sirius annotation valable. '0.0' as default.

#Literature_component

LC_component = True               # LC will be calculated

max_comp_reported_sp = 10          # max number of compounds reported at species level, more than this value, the plant is considered less interesting
max_comp_reported_g = 50         # max number of compounds reported at genus level,more than this value, the plant is considered less interesting
max_comp_reported_f = 500           # max number of compounds reported at genus level,more than this value, the plant is considered less interesting

#weight for each taxonomic level 
ws = 1
wg = 1
wf = 1 

#Similarity_component

SC_component = True                # SC will be calculated

#Class_component

CC_component =  True              # CC will be calculated
min_class_confidence = 0.8       #cut-off filter for considering a sirius class valable. It is used in combination with min_recurrence.
min_recurrence = 5              # minimum recurrence of a chemical class to consider it acceptable

#specify the weight to modulate each component 
w1 = 1           # 1 means the value itself is taken into account. A 0.5 means onle half of the calculated value is taken into account
w2 = 1
w3 = 1
w4 = 1


### Load Metadata from individual files

In [3]:
metadata_df = get_metadata_ind_files(repository_path)
metadata_df.head()

100%|██████████| 17/17 [00:00<00:00, 114.08it/s]


,sample_id,sample_type,sample_substance_name,organism_kingdom,organism_phylum,organism_class,organism_order,organism_family,organism_genus,organism_species,...,pos_injection_date,bio_leish_donovani_10ugml_inhibition,bio_leish_donovani_2ugml_inhibition,bio_tryp_brucei_rhodesiense_10ugml_inhibition,bio_tryp_brucei_rhodesiense_2ugml_inhibition,bio_tryp_cruzi_10ugml_inhibition,bio_l6_cytotoxicity_10ugml_inhibition,sample_filename_neg,neg_injection_date,massive_id
0,VGF138_A01,qc,qc_mix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017-10-27,NaN,NaN,NaN,NaN,NaN,NaN,VGF138_A01_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A02,sample,V111819GP-01,Plantae,Tracheophyta,Magnoliopsida,Saxifragales,Paeoniaceae,Paeonia,Paeonia suffruticosa,...,2017-10-27,16.0,10.2,4.8,0.0,8.7,4.2,VGF138_A02_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A03,sample,V111988GP-01,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Scrophulariaceae,Buddleja,Buddleja officinalis,...,2017-10-27,36.7,12.2,9.5,6.0,0.0,0.5,VGF138_A03_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A04,sample,V112020GP-01,Plantae,Tracheophyta,Magnoliopsida,Rosales,Moraceae,Ficus,Ficus tikoua,...,2017-10-27,20.9,10.4,23.6,4.7,1.1,22.5,VGF138_A04_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A05,sample,V112033GP-01,Plantae,Tracheophyta,Magnoliopsida,Solanales,Solanaceae,Nicandra,Nicandra physalodes,...,2017-10-27,5.7,6.7,14.1,7.7,2.5,7.5,VGF138_A05_neg.mzXML,2017-10-27,MSV000087728


In [4]:
#if you need to create an unique identifier column like Species|part, use as model the followin line. IF the colum is PRESENT, then don't run it.
metadata_df['organism_sppart'] = metadata_df[species_column]+ "|" + metadata_df[organe_column].map(str)
metadata_df.head(5)

,sample_id,sample_type,sample_substance_name,organism_kingdom,organism_phylum,organism_class,organism_order,organism_family,organism_genus,organism_species,...,bio_leish_donovani_10ugml_inhibition,bio_leish_donovani_2ugml_inhibition,bio_tryp_brucei_rhodesiense_10ugml_inhibition,bio_tryp_brucei_rhodesiense_2ugml_inhibition,bio_tryp_cruzi_10ugml_inhibition,bio_l6_cytotoxicity_10ugml_inhibition,sample_filename_neg,neg_injection_date,massive_id,organism_sppart
0,VGF138_A01,qc,qc_mix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VGF138_A01_neg.mzXML,2017-10-27,MSV000087728,NaN
0,VGF138_A02,sample,V111819GP-01,Plantae,Tracheophyta,Magnoliopsida,Saxifragales,Paeoniaceae,Paeonia,Paeonia suffruticosa,...,16.0,10.2,4.8,0.0,8.7,4.2,VGF138_A02_neg.mzXML,2017-10-27,MSV000087728,Paeonia suffruticosa|leaves
0,VGF138_A03,sample,V111988GP-01,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Scrophulariaceae,Buddleja,Buddleja officinalis,...,36.7,12.2,9.5,6.0,0.0,0.5,VGF138_A03_neg.mzXML,2017-10-27,MSV000087728,Buddleja officinalis|leaves
0,VGF138_A04,sample,V112020GP-01,Plantae,Tracheophyta,Magnoliopsida,Rosales,Moraceae,Ficus,Ficus tikoua,...,20.9,10.4,23.6,4.7,1.1,22.5,VGF138_A04_neg.mzXML,2017-10-27,MSV000087728,Ficus tikoua|multiple
0,VGF138_A05,sample,V112033GP-01,Plantae,Tracheophyta,Magnoliopsida,Solanales,Solanaceae,Nicandra,Nicandra physalodes,...,5.7,6.7,14.1,7.7,2.5,7.5,VGF138_A05_neg.mzXML,2017-10-27,MSV000087728,Nicandra physalodes|fruits


In [5]:
col_id_unique = filename_header #'organism_sppart'  #column containing an unique identifier for each sample, like Species_plantpart, Species_solvent. It could be the filename

# Start calculation the diferent components

# Annotation Component (FC)

#### AC.1. Process, clean and merge the quant tables, sirius and isdb annotations

In [6]:
ind_quant_table_full(repository_path, ionization_mode, data_process_origin, file_extention, use_ion_identity, min_score_final, min_ConfidenceScore, min_ZodiacScore)

100%|██████████| 17/17 [00:00<00:00, 17.53it/s]

Result are in : /mnt/c/Users/quirosgu/Desktop/Indiv_PF1600/results/VGF138_B04_pos_quant_annotations.tsv


#### AC.2. Calculate the annotation rate of each sample

In [7]:
AC = annotation_component(repository_path, ionization_mode, intensity_filter, quantile_filter, min_threshold, quantile_threshold, file_extention, filename_header)
AC.head()

100%|██████████| 17/17 [00:00<00:00, 109.49it/s]


,sample_id,initial_features,features_after_filtering,Annot_features_after_filtering,AC
0,VGF138_A02,343,343,148,0.6
1,VGF138_A03,514,514,238,0.5
2,VGF138_A04,498,498,254,0.5
3,VGF138_A05,697,697,298,0.6
4,VGF138_A06,522,522,226,0.6


# Literature Component (LC)


#### LC.1. LC computation

In [8]:
LC = literature_component(LC_component, metadata_df, filename_header, species_column, genus_column, family_column, max_comp_reported_sp, max_comp_reported_g, max_comp_reported_f, ws, wg, wf)
LC.head()

,sample_id,organism_family,organism_genus,organism_species,Reported_comp_Species,Reported_comp_Genus,Reported_comp_Family,LC
0,VGF138_A02,Paeoniaceae,Paeonia,Paeonia suffruticosa,271.0,1058.0,1058,0.49624
1,VGF138_A03,Scrophulariaceae,Buddleja,Buddleja officinalis,133.0,557.0,2301,0.70958
2,VGF138_A04,Moraceae,Ficus,Ficus tikoua,0.0,895.0,6850,0.68400
3,VGF138_A05,Solanaceae,Nicandra,Nicandra physalodes,6.0,6.0,13616,0.72048
4,VGF138_A06,Polygalaceae,Asemeia,Asemeia extraaxillaris,0.0,74.0,1434,0.95652


# Similarity component (SC)

#### SC.1. Calculate MEMO matrix from individual files

In [9]:
#calculate MEMO matrix from individual files
metric_df = calculate_memo_matrix_ind_files(repository_path, spectra_suffix, filename_header)
#metric_df.head()

100%|██████████| 16/16 [00:09<00:00,  1.60it/s]


Computing MEMO matrix from unaligned samples took: 10.46875 seconds


../src/SC.py:99: FutureWarning: The default value of regex will change from True to False in a future version.
  memo_unaligned.memo_matrix.index = memo_unaligned.memo_matrix.index.str.replace(spectra_suffix, "")


In [10]:
#remove experimental controls and blancks
list_of_strings_for_QC_Blank_filter = ['blank', 'qc'] #erase all the blanks and QC's - Change the string as needed
column_to_use_for_filtering = sampletype_header #this information should be included in the metadata table
metric_df= drop_samples_based_on_string_ind(metric_df,metadata_df, filename_header, sampletype_header,'metric_df', list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)
#metric_df.head()


(16, 30779)
(13, 30779)


#### SC.2. SC calculation

In [11]:
SC = similarity_component(metric_df, SC_component, filename_header)
SC.head(10)

,sample_id,anomaly_IF,anomaly_LOF,anomaly_OCSVM,SC
0,VGF138_A02,1,1,1,0
1,VGF138_A03,1,1,1,0
2,VGF138_A04,1,1,1,0
3,VGF138_A05,1,1,1,0
4,VGF138_A06,1,1,1,0
5,VGF138_A07,1,-1,-1,1
6,VGF138_A08,1,1,1,0
7,VGF138_A09,1,-1,1,1
8,VGF138_A10,1,1,1,0
9,VGF138_A11,1,1,1,0


# Class component (CC)

In [28]:
CC= class_component_ind_files_PF1600(CC_component, repository_path, min_class_confidence, metadata_df, filename_header, species_column, genus_column, family_column)
CC.head()

100%|██████████| 17/17 [00:00<00:00, 115.05it/s]


,sample_id,NPC#class,organism_species,organism_genus,organism_family,Chemical_class_reported_in_species,Chemical_class_reported_in_genus,New_CC_in_sp,New_CC_in_genus,CCs,CCg,CC
0,VGF138_A02,"{Flavones, Cyanogenic glycosides, Cholestane s...",Paeonia suffruticosa,Paeonia,Paeoniaceae,{Bauerane triterpenoids|Ursane and Taraxastane...,{Bauerane triterpenoids|Ursane and Taraxastane...,"{Flavones, Neutral glycosphingolipids, Cyanoge...","{Flavones, Neutral glycosphingolipids, Cyanoge...",0.5,0.5,1.0
1,VGF138_A03,"{Flavones, Lanostane, Tirucallane and Euphane ...",Buddleja officinalis,Buddleja,Scrophulariaceae,"{Flavones, Iridoids monoterpenoids, Acyclic mo...","{Flavones, Iridoids monoterpenoids, Acyclic mo...","{Cyclic peptides, Cholestane steroids, Nitro f...","{Cyclic peptides, Cholestane steroids, Nitro f...",0.5,0.5,1.0
2,VGF138_A04,"{Flavones, Lanostane, Tirucallane and Euphane ...",Ficus tikoua,Ficus,Moraceae,nothing in DB,nothing in DB,nothing in DB,nothing in DB,0.0,0.0,1.0
3,VGF138_A05,"{Cyclic peptides, Cholestane steroids, Limonoi...",Nicandra physalodes,Nicandra,Solanaceae,"{Cycloartane triterpenoids|Lanostane, Tirucall...","{Cycloartane triterpenoids|Lanostane, Tirucall...","{Neutral glycosphingolipids, Cyclic peptides, ...","{Neutral glycosphingolipids, Cyclic peptides, ...",0.5,0.5,1.0
4,VGF138_A06,"{Flavones, Cyclic peptides, Fatty acyl glycosi...",Asemeia extraaxillaris,Asemeia,Polygalaceae,nothing in DB,nothing in DB,nothing in DB,nothing in DB,0.0,0.0,1.0


# Priority rank Results

In [38]:
PR = priority_score_ind(AC, LC, SC, CC, LC_component, SC_component, CC_component, w1, w2, w3, w4, filename_header)
PR.head()

,sample_id,initial_features,features_after_filtering,Annot_features_after_filtering,AC,LC,Reported_comp_Species,Reported_comp_Genus,Reported_comp_Family,SC,CCs,CCg,CC,New_CC_in_sp,New_CC_in_genus,PS
0,VGF138_A02,343,343,148,0.6,0.49624,271.0,1058.0,1058,0,0.5,0.5,1.0,"{Flavones, Neutral glycosphingolipids, Cyanoge...","{Flavones, Neutral glycosphingolipids, Cyanoge...",2.09624
1,VGF138_A03,514,514,238,0.5,0.70958,133.0,557.0,2301,0,0.5,0.5,1.0,"{Cyclic peptides, Cholestane steroids, Nitro f...","{Cyclic peptides, Cholestane steroids, Nitro f...",2.20958
2,VGF138_A04,498,498,254,0.5,0.68400,0.0,895.0,6850,0,0.0,0.0,1.0,nothing in DB,nothing in DB,2.18400
3,VGF138_A05,697,697,298,0.6,0.72048,6.0,6.0,13616,0,0.5,0.5,1.0,"{Neutral glycosphingolipids, Cyclic peptides, ...","{Neutral glycosphingolipids, Cyclic peptides, ...",2.32048
4,VGF138_A06,522,522,226,0.6,0.95652,0.0,74.0,1434,0,0.0,0.0,1.0,nothing in DB,nothing in DB,2.55652


In [ ]:
Cyt_format_visualization = Cyt_format(col_id_unique)

### Display results

In [ ]:
#Show the results in an interactive way
def selection_changed(selection):
    return PR.iloc[selection]
interact(selection_changed, selection=lineup_widget.LineUpWidget(PR));

interactive(children=(LineUpWidget(value=[], description='selection', layout=Layout(align_self='stretch', heig…